In [ ]:
import numpy as np
import pandas as pd
import math
yita = 0.01
STEPS = 1000

In [ ]:
df = pd.read_csv('watermelon.csv')

In [ ]:
"""
input:
    dataframe:df数据集
    test_ratio:验证集的比例
output:
    训练集、测试集
"""
def trainTestSplit(dataframe, test_ratio = 0.3):
    row, col = dataframe.shape
    randomIndex = np.random.permutation(row)
#     print(randomIndex)

    trainNum = int(row * (1-test_ratio))
    trainIndex = randomIndex[:trainNum]
    testIndex = randomIndex[trainNum:]
    train_data = dataframe.ix[trainIndex]
    test_data = dataframe.ix[testIndex]
    
    return train_data, test_data

In [ ]:
train_data, test_data = trainTestSplit(df,test_ratio = 0.2)
# print(train_data)
# print(test_data)
train_data.index

In [ ]:
data_x = train_data[['density','ratio_suger']].values
# ind = train_data.index[train_data['label']==0]
# print(ind)
# train_data.loc[ind,'label'] = -1
print(train_data)
data_y = train_data['label'].values 
data_y = to_categorical(data_y)
test_x = test_data[['density','ratio_suger']].values
test_y = test_data['label'].values
test_y = to_categorical(test_y)
print(test_y)

In [ ]:
def to_categorical(y, num_classes=None):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
#     print(categorical)
#     output_shape = input_shape + (num_classes,)
# #     print(input_shape)
# #     print(output_shape)
#     categorical = np.reshape(categorical, output_shape)
    
    return categorical

In [ ]:
#print(test_y.shape)
#print((4,9)+(2,3))
# arr = np.zeros((3,3))
# arr[[0,1],0] = 1
# print(arr)
to_categorical(test_y)

In [ ]:
def inference(w1, b1, w2, b2, x):
    fctemp = np.dot(x,w1) + b1
    fc1 = 1 / (1 + np.exp(-fctemp))
    fctemp = np.dot(fc1, w2) + b2
    fc2 = 1 / (1 + np.exp(-fctemp))
    
    return fc1,fc2

In [ ]:
a = np.array([[1,2],[3,4]])
a[1][1]

In [ ]:
"""
构建神经网络
"""
w1 = np.random.rand(2,2)
b1 = np.random.rand(2)
w2 = np.random.rand(2,2)
b2 = np.random.rand(2)

row, col = data_x.shape

for step in range(STEPS):
    E = 0
    for i in range(row):
        x = data_x[i]
        y = data_y[i]
        fc1, y_ = inference(w1, b1, w2, b2, x)
#         print('fc1:',fc1, 'type:fc1:', type(fc1), 'y_', y_)
#         print('y:', y, 'y_:', y_)
        Ek = np.sum(np.power((y - y_), 2)) / 2
    #     print(i, y_)
    #     print(i,Ek)
        E += Ek
        g = y_ * (1 - y_) * (y - y_)
        e = fc1 * (1 - fc1) * np.dot(w2 , g)
#         print(i, e)
        w1 += yita * np.mat(x).T * np.mat(e)
        b1 += -yita * e
        w2 += yita * np.mat(fc1).T * np.mat(g)
        b2 += -yita * g
    if step % 10 == 0:
#         _, y_ = inference(w1, b1, w2, b2, )
        
        print('After {} training steps, E is {}.'.format(step, E))


In [ ]:
"""
构建神经网络
"""
w1 = np.random.rand(2,2)
b1 = np.random.rand(2)
w2 = np.random.rand(2,2)
b2 = np.random.rand(2)

row, col = data_x.shape

for step in range(STEPS):
    E = 0
    for i in range(row):
        x = data_x[i]
        y = data_y[i]
        fc1, y_ = inference(w1, b1, w2, b2, x)
#         print('fc1:',fc1, 'type:fc1:', type(fc1), 'y_', y_)
        print('y:', y, 'y_:', y_)
        Ek = np.sum(np.power((y - y_), 2)) / 2
    #     print(i, y_)
    #     print(i,Ek)
        E += Ek
        g = []
        for j in range(len(y_)):
            gj = y_[j] * (1 - y_[j]) * (y[j] - y_[j])
            b2[j] += -yita * gj
            g.append(gj)
        e = []
        for h in range(len(fc1)):
            eh = fc1[h] * (1 - fc1[h]) * np.dot(w2[h] , np.array(g))
            b1[h] += -yita * eh 
            e.append(eh)
        
        print(i, e)
        for i in range(len(x)):
            for h in range(len(fc1)):
                w1[i][h] += yita * e[h] * x[i]
        for h in range(len(fc1)):
            for j in range(len(y_)):
                w2[h][j] += yita * g[j] * fc1[h] 
    if step % 10 == 0:
#         _, y_ = inference(w1, b1, w2, b2, )
        
        print('After {} training steps, E is {}.'.format(step, E))


In [ ]:
traindf = df[['根蒂','脐部']]
ind = traindf.index[traindf['脐部'] == '凹陷']
traindf.loc[ind,'脐部'] = 0
ind = traindf.index[traindf['脐部'] == '稍凹']
traindf.loc[ind,'脐部'] = 1
ind = traindf.index[traindf['脐部'] == '平坦']
traindf.loc[ind,'脐部'] = 2
ind = traindf.index[traindf['根蒂'] == '蜷缩']
traindf.loc[ind,'根蒂'] = 0
ind = traindf.index[traindf['根蒂'] == '稍蜷']
traindf.loc[ind,'根蒂'] = 1
ind = traindf.index[traindf['根蒂'] == '硬挺']
traindf.loc[ind,'根蒂'] = 2
data_x = traindf.values
data_y = df['label'].values
data_y = to_categorical(data_y)